## Imports

In [74]:
from numpy import array
from math import atan
from fastjet import PseudoJet, JetDefinition, ClusterSequence, antikt_algorithm
import awkward as ak
import vector
import uproot
vector.register_awkward()

In [47]:
!pip install fastjet
!pip install vector
!pip install uproot

## PseudoJet
1. **PseudoJet** is a class in the **FastJet** library.
2. **PseudoJet** provides a jet object with a four-momentum and some internal indices to situate it in the context of a jet-clustering sequence.
3. **PseudoJet** is basically an object containing information of a particle or a bunch of particles, the information it contains is the four-momentum of the particles.

In [82]:
jet_1 = PseudoJet(1.1, 2.2, 3.5, 4.5) # Creating an object containing 4-momentum of a particle using PseudoJet() function
print("Energy component :", jet_1.E()) # This returns energy component of the 4-momentum, jet_1.e().
print("Momentum x-component :", jet_1.px()) # This returns x-component of the 3-momentum.
print("Momentum y-component :", jet_1.py()) # This returns y-component of the 3-momentum.
print("Momentum z-component :", jet_1.pz()) # This returns z-component of the 3-momentum.
print("Azimuth of the particle :", jet_1.phi()) # This returns azimuthal angle in the range 0 to 2pi.
# Checking the result with the definition of azimuthal angle
a = jet_1.px()
b = jet_1.py()
c = b/a
print(atan(c)) # Note that this gives the same result as in previous with phi() function.
print("Standard azimuth :", jet_1.phi_std()) # This returns azimuthal angle in the range -pi to pi.
print("Rapidity :", jet_1.rap()) # This returns the rapidity, jet_1.rapidity().
print("Pseudo-rapidity :", jet_1.pseudorapidity()) # This returns the pseudo-rapidity, jet_1.eta().
print("Squared transverse momentum :", jet_1.pt2()) # This returns the squared transverse momentum, jet_1.perp2().
print("Transverse momentum :", jet_1.pt()) # This returns the transverse momentum, jet_1.perp().
print("Squared invariant mass :", jet_1.m2()) # This returns squared invariant mass.
print("Invariant mass :", jet_1.m()) # This returns invariant mass.
print("Squared transverse mass :", jet_1.mt2()) # This returns the squared transverse mass, jet_1.mperp2().
print("Transverse mass :", jet_1.mt()) # This returns the transverse mass, jet_1.mperp().
jet_2 = PseudoJet(1.1, 2.2, 3.5, 4.5)
jet_3 = PseudoJet(1.1, 2.2, 3.5, 0.0)
jet_4 = PseudoJet(0, 0, 0, 0)
# The equality testing operators
print(jet_1 == jet_2) # This returns true if the two jets have identical 4-momenta, structural information and user information.
print(jet_2 == jet_3)
print(jet_4 == 0) # This returns true if all the components of the 4-momentum are zero.
print(jet_3 != 0) # This returns true if any of the components of the 4-momentum is not zero.
jet_1.set_user_index(2) # Setting the user_index, intended to allow the user to label the object (default is -1)
jet_1.user_index() # This returns the user_index.
jet_2.user_index()
jet_1.reset(1.1, 2.2, 3.5, 4.5) # Reset the 4-momentum according to the supplied components, put the user and history indices
# and user info back to their default values (-1, unset)
jet_1.user_index()
print("Old jet 1 :", jet_1)
jet_1.reset_momentum(1, 2, 3, 4) #  Reset just the 4-momentum according to the supplied components, all other information is left unchanged
print("New jet 1 :", jet_1)
print("User index :", jet_1.user_index())
jet_sum = jet_2 + jet_3 # Addition of two PseudoJet objects
print("Added jet (23) :", jet_sum)
jet_diff = jet_2 - jet_3 # Subtraction of two PseudoJet objects
print("Jet diff. (23) :", jet_diff)
# The +, -, * and / operators are defined, with +, - acting on pairs of PseudoJet.
# The analogous +=, etc., operators, are also defined.

Energy component : 4.5
Momentum x-component : 1.1
Momentum y-component : 2.2
Momentum z-component : 3.5
Azimuth of the particle : 1.1071487177940904
1.1071487177940904
Standard azimuth : 1.1071487177940904
Rapidity : 1.0397207708399179
Pseudo-rapidity : 1.1512508050083592
Squared transverse momentum : 6.050000000000001
Transverse momentum : 2.459674775249769
Squared invariant mass : 1.9499999999999993
Invariant mass : 1.3964240043768938
Squared transverse mass : 8.0
Transverse mass : 2.8284271247461903
True
False
True
True
Old jet 1 : [1.100000, 2.200000, 3.500000, 4.500000]
New jet 1 : [1.000000, 2.000000, 3.000000, 4.000000]
User index : -1
Added jet (23) : [2.200000, 4.400000, 7.000000, 4.500000]
Jet diff. (23) : [0.000000, 0.000000, 0.000000, 4.500000]


In [84]:
# Sorting of PsudoJet objects
particles = [PseudoJet(0.7283822313,0.1094889185 ,-0.1754670827  ,-0.6843130641),
             PseudoJet(6.7420362761,-1.3395280577,-1.0677537402,-6.4527380155),
             PseudoJet(0.9647588756,-0.4610425267 ,-0.0168193957 ,-0.8357253135),
             PseudoJet(6.5019392609 ,-0.8586330562 ,-2.0252604440 ,-6.1169276419),
             PseudoJet(0.4622264166,0.1506846470 ,-0.2812835366 ,-0.3038867955)]

particles.sort(key = lambda jet:jet.pt()) # The particles are sorted based on increasing transverse momentum.

print("Particles transverse momentum in increasing order :")

for i in range(5) :
  print(particles[i].pt())

Particles transverse momentum in increasing order :
0.48616779309657215
0.736565338682093
1.0692613803345263
6.558388893442672
6.873819066982626


## Jet definition
1. The class **JetDefinition** contains a specification of how jet
clustering is to be performed.
2. The **JetDefinition** class takes varied number of arguments, the first argument is always the type of algorithm, the number of rest of the arguments depends on how many parameters the given algorithm requires.
3. According to the Les Houches convention, a 'jet definition' should include the jet algorithm name, its parameters (often the radius R) and the recombination scheme.

In [86]:
jetdef = JetDefinition(antikt_algorithm, 0.6)

## JetAlgorithms
The **JetDefinition** class takes **JetAlgorithms** as arguments. In the above example we have chosen the **anti-$k_t$** algorithm. The list of algorithms is as following:


* **kt_algorithm** : The longitudinally invariant $k_t$ algorithm with distance measures

	\begin{align*}
	d_{ij} &= \min(p_{Ti}^2,p_{Tj}^2) \frac{\Delta R_{ij}^2}{R^2}, \\
	d_{iB} &= p_{Ti}^2.
	\end{align*}
* **cambridge_algorithm** : The longitudinally invariant variant of the cambridge algorithm (aka Aachen algoithm) with distance measures
	\begin{align*}
	d_{ij} &= \frac{\Delta R_{ij}^2}{R^2}, \\
	d_{iB} &= 1.
	\end{align*}
* **antikt_algorithm** : Like the $k_t$ but with distance measures
	\begin{align*}
	d_{ij} &= \min(p_{Ti}^{-2},p_{Tj}^{-2}) \frac{\Delta R_{ij}^2}{R^2}, \\
	d_{iB} &= p_{Ti}^{-2}.
	\end{align*}
* **genkt_algorithm** : The generalized $k_t$ algorithm but with distance measures
	\begin{align*}
    d_{ij} &= \min(p_{Ti}^{2p},p_{Tj}^{2p}) \frac{\Delta R_{ij}^2}{R^2}, \\
    d_{iB} &= p_{Ti}^{2p}
	\end{align*}

  where **p = extra_param()**.\
  Special cases are for : $p=1$ (gives the $k_t$ algorithm), $p=0$ (gives the Cambridge/Aachen algorithm), and $p=-1$ (gives the anti-$k_t$ algorithm).



## Recombination schemes
1. For merging particles (i.e **PseudoJet**'s) during the clustering procedure, one must specify how to combine the momenta.
2. The recombination scheme is set by stating **RecombinationScheme** in the **JetDefinition**, and it is related to the choice of how to recombine the 4-momenta of **PseudoJet**'s during the clustering procedure.
3. The default in **FastJet** is the **E_scheme**, where the four components of two 4-vectors are simply added. This scheme is used when no explicit choice is made in the constructor.

## Jet clustering
For jet clustering, **FastJet** has a class **ClusterSequence** that carries out jet-clustering and provides access to the final jets. For clustering of particles, the class **ClusterSequence** requires the information of particles and the algorithm for clustering process. This is how jet clustering is done.

In [87]:
particles = [PseudoJet(0.7283822313, 0.1094889185, -0.1754670827, -0.6843130641),
             PseudoJet(6.7420362761, -1.3395280577, -1.0677537402, -6.4527380155),
             PseudoJet(0.9647588756, -0.4610425267 , -0.0168193957, -0.8357253135),
             PseudoJet(6.5019392609, -0.8586330562, -2.0252604440, -6.1169276419),
             PseudoJet(0.4622264166, 0.1506846470, -0.2812835366, -0.3038867955)]

cluster = ClusterSequence(particles, jetdef)

In [88]:
inc_jets = cluster.inclusive_jets() # Jets obtained from inclusive clustering algorithm

for i in range(len(inc_jets)) :
  print("px : {px}, py : {py}, pz : {pz}, E : {E}".format(px=inc_jets[i].px(), py = inc_jets[i].py(), pz = inc_jets[i].pz(), E = inc_jets[i].E())) # This prints 4-momentum of all the jets.


px : 0.4622264166, py : 0.150684647, pz : -0.2812835366, E : -0.3038867955
px : 14.9371166439, py : -2.5497147221, pz : -3.2853006626, E : -14.089704034999999


In [103]:
# Using Awkward arrays to define 4-momentum of particles
array = ak.Array(
    [
        [
            {"px": 0.45417, "py": 0.32728, "pz": -0.23645, "E": 0.15539, "charge": 1},
            {"px": 1.89939, "py": -0.78538, "pz": -0.78105, "E": -1.53662, "charge": 1},
            {"px": 3.3957985464, "py": 3.20144, "pz": -0.31280, "E": -0.55188, "charge": 1},
        ],
        [],
        [
            {"px": 2.95593, "py": -0.35361, "pz": 0.62956, "E": -2.86073, "charge": 1},
            {"px": 4.33707, "py": 0.53039, "pz": 1.47772, "E": -3.93265, "charge": 1},
            {"px": 0.32284, "py": 0.06157, "pz": 0.12261, "E": -0.25641, "charge": 1},
            {"px": 0.32114, "py": 0.01579, "pz": 0.01635, "E": -0.32380, "charge": 1},
        ],
        [
            {"px": 9.7436, "py": -0.0011, "pz": 0.238, "E": -9.7392, "charge": 1}
        ],
    ],
)

In [10]:
cluster = ClusterSequence(array, jetdef)
cluster.inclusive_jets().to_list() # A vector of jets

[[{'px': 0.45417, 'py': 0.32728, 'pz': -0.23645, 'E': 0.15539},
  {'px': 1.89939, 'py': -0.78538, 'pz': -0.78105, 'E': -1.53662},
  {'px': 3.3957985464, 'py': 3.20144, 'pz': -0.3128, 'E': -0.55188}],
 [],
 [{'px': 7.93698,
   'py': 0.25414000000000003,
   'pz': 2.2462400000000002,
   'E': -7.373590000000001}],
 [{'px': 9.7436, 'py': -0.0011, 'pz': 0.238, 'E': -9.7392}]]

In [11]:
output = cluster.constituents() # Constituents of each jet
output.to_list()

[[[{'px': 0.45417, 'py': 0.32728, 'pz': -0.23645, 'E': 0.15539, 'charge': 1}],
  [{'px': 1.89939,
    'py': -0.78538,
    'pz': -0.78105,
    'E': -1.53662,
    'charge': 1}],
  [{'px': 3.3957985464,
    'py': 3.20144,
    'pz': -0.3128,
    'E': -0.55188,
    'charge': 1}]],
 [],
 [[{'px': 2.95593, 'py': -0.35361, 'pz': 0.62956, 'E': -2.86073, 'charge': 1},
   {'px': 4.33707, 'py': 0.53039, 'pz': 1.47772, 'E': -3.93265, 'charge': 1},
   {'px': 0.32284, 'py': 0.06157, 'pz': 0.12261, 'E': -0.25641, 'charge': 1},
   {'px': 0.32114, 'py': 0.01579, 'pz': 0.01635, 'E': -0.3238, 'charge': 1}]],
 [[{'px': 9.7436, 'py': -0.0011, 'pz': 0.238, 'E': -9.7392, 'charge': 1}]]]

Now let's do the jet clustering with more number of particles as the input data.

In [16]:
from google.colab import drive
drive.mount('/content/drive')
data_file = "/content/drive/MyDrive/JetClass_example_100k.root"
tree = uproot.open(data_file)["tree"]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
table = tree.arrays()

In [40]:
# Construct a Lorentz 4-vector from the (px, py, pz, energy) arrays
p4 = vector.zip({"px": table["part_px"], "py": table["part_py"], "pz": table["part_pz"], "energy": table["part_energy"]})

In [93]:
p4 # Each element consists of Lorentz vectors of particles.

<MomentumArray4D [[{x: -125, y: 64.3, ...}, ...], ...] type='100000 * var *...'>

In [92]:
p4[0][0] # Accessing Lorentz vector of one of the particle

<MomentumRecord4D {x: -125, y: 64.3, z: -36.1, ...} type='Momentum4D[x: flo...'>

In [90]:
# Creating a list of Lorentz vectors of the particles
N = 10
particles = []
for i in range(N) :
  num_jet_part = table["jet_nparticles"][i]
  for j in range(int(num_jet_part)) :
    particles.append(p4[i][j])

In [91]:
particles[0] # This returns a Lorentz vector of the first particle in the list.

<MomentumRecord4D {x: -125, y: 64.3, z: -36.1, ...} type='Momentum4D[x: flo...'>

In [94]:
num_jet_part = sum([table["jet_nparticles"][i] for i in range(10)])
num_jet_part # This returns total number of particles in the list.

389.0

In [99]:
inp_data = ak.Array(particles, with_name = "Momentum4D")
cluster = ClusterSequence(inp_data, jetdef)
cluster.inclusive_jets() # This returns list a of jets, each specifying the 4-momentum of the jet. A total of 17 jets is obtained.

<MomentumArray4D [{px: 1.32, py: -0.946, ...}, ...] type='17 * Momentum4D[p...'>

In [100]:
cluster.constituents() # Constituents of each jet
# The total number of constituents must be equal to the total number of particles.

<MomentumArray4D [[{px: 1.32, py: -0.946, ...}], ...] type='17 * var * Mome...'>

In [102]:
sum([len(cluster.constituents()[i]) for i in range(17)]) # The result is same as the total number of particles.

389